### Set working directory

In [12]:
%cd ~/theta_methyl/

/Users/bartosz/theta_methyl


### Download FASTQ file

In [11]:
!wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR617/001/SRR6176401/SRR6176401.fastq.gz

--2023-03-20 22:11:01--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR617/001/SRR6176401/SRR6176401.fastq.gz
           => 'SRR6176401.fastq.gz'
Resolving ftp.sra.ebi.ac.uk... 193.62.193.138
Connecting to ftp.sra.ebi.ac.uk|193.62.193.138|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /vol1/fastq/SRR617/001/SRR6176401 ... done.
==> SIZE SRR6176401.fastq.gz ... 1081731478
==> PASV ... done.    ==> RETR SRR6176401.fastq.gz ... done.
Length: 1081731478 (1.0G) (unauthoritative)

SRR6176401.fastq.gz 100%[===================>]   1.01G  2.28MB/s    in 7m 13s  

2023-03-20 22:18:15 (2.38 MB/s) - 'SRR6176401.fastq.gz' saved [1081731478]



### Download reference genome (let's download only one chromosome to make the analysis faster)
###### https://ftp.ensemblgenomes.ebi.ac.uk/pub/plants/release-56/fasta/arabidopsis_thaliana/dna/Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz

In [31]:
!wget https://ftp.ensemblgenomes.ebi.ac.uk/pub/plants/release-56/fasta/arabidopsis_thaliana/dna/Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz

--2023-03-21 20:53:34--  https://ftp.ensemblgenomes.ebi.ac.uk/pub/plants/release-56/fasta/arabidopsis_thaliana/dna/Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz
Resolving ftp.ensemblgenomes.ebi.ac.uk... 193.62.193.141
Connecting to ftp.ensemblgenomes.ebi.ac.uk|193.62.193.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9211107 (8.8M) [application/x-gzip]
Saving to: 'Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz'

Arabidopsis_thalian 100%[===================>]   8.78M  1.24MB/s    in 7.2s    

2023-03-21 20:53:41 (1.23 MB/s) - 'Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz' saved [9211107/9211107]



In [53]:
!head <(gzcat Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz)

>1 dna:chromosome chromosome:TAIR10:1:1:30427671:1 REF
CCCTAAACCCTAAACCCTAAACCCTAAACCTCTGAATCCTTAATCCCTAAATCCCTAAAT
CTTTAAATCCTACATCCATGAATCCCTAAATACCTAATTCCCTAAACCCGAAACCGGTTT
CTCTGGTTGAAAATCATTGTGTATATAATGATAATTTTATCGTTTTTATGTAATTGCTTA
TTGTTGTGTGTAGATTTTTTAAAAATATCATTTGAGGTCAATACAAATCCTATTTCTTGT
GGTTTTCTTTCCTTCACTTAGCTATGGATGGTTTATCTTCATTTGTTATATTGGATACAA
GCTTTGCTACGATCTACATTTGGGAATGTGAGTCTCTTATTGTAACCTTAGGGTTGGTTT
ATCTCAAGAATCTTATTAATTGTTTGGACTGTTTATGTTTGGACATTTATTGTCATTCTT
ACTCCTTTGTGGAAATGTTTGTTCTATCAATTTATCTTTTGTGGGAAAATTATTTAGTTG
TAGGGATGAAGTCTTTCTTCGTTGTTGTTACGCTTGTCATCTCATCTCTCAATGATATGG


##### Imagine this file contains the whole reference genome. How to replace the header of FASTA file by presenting only:
##### >1
##### ...
##### >2
##### ...
##### etc.? Replace the header by short chromosome name